### 어텐션

- 어텐션층의 공간복잡도는 약 $n^2$ 이다<br>
$n$ : 시퀀스 길이

### 계산이 중요하다.
- 규모의 법칙
    - N, C, D 를 동시에 증가시키는 것이 더 나은 모델을 만드는 데 생산적이다.
        - N: 모델 크기
        - C: 컴퓨팅 예산
        - D: 데이터 셋 크기

- 거듭 제곱 법칙(power law)
    - 언어 모델의 성능은 거듭 제곱 법칙을 따른다.
        - 테스트 손실 $L(X) \sim \left( \frac{1}{X} \right)^{\alpha}$

- 샘플 효율성
    - 대규모 모델은 적은 훈련횟수로도 소규모 모델과 동일한 성능을 낸다.
        - 이미지, 비디오, 수학 문제

### 규모확장의 어려움
- 인프라
    - GPU 가 여럿, 노드가 수백 수천 개를 관리하는 일은 쉽지 않다.
    - 대규모 분산 을 능숙하게 해야한다.
- 비용
    - GPT-3 모델 하나 훈련시에 수백만 달러가 들기도 한다.
    - GPT-4 훈련에 1억달러(약 1400억) 이상이 들었다
- 고품질의 데이터셋
    - 전처리의 어려움
    - 고품질의 데이터셋인지 확인의 어려움
    - 성차별, 편향 제어 방법 모색
- 모델 평가
    - 언어 모델 평가에 대한 시간과 자원
    - 유해성 검증
- 배포
    - 대규모 모델을 서빙하는데 어려움
    - 양자, 가지치기, 정제등의 모델 압축 및 경량화 기술이 있지만 충분치 않다.
        - Openai Api, Accelerated Inference Api

- BigScience, EleutherAI
    - 오픈소스 연구 단체 들
    - 여러 오픈소스 모델을 훈련하였고 배포함

### 다양한 어텐션 연구방향 https://arxiv.org/abs/2106.04554 (2021)

- 희소 어텐션
    - ![sparse attn](images/11_01.png)
    - ![model](images/11_02.png)
- 선형 어텐션
    - ![linear attn](images/11_03.png)
        - 기존 식 : $$ \text{softmax}(\frac{Q \times K^T}{\sqrt d_k}) \times V$$
        - 선형 식 : $$ \phi(Q) \times  (\phi(K)^T \times V)$$ 여기서 $\phi$ 는 특성 변환 함수
            - 특성 변환 함수 예시(피쳐 매핑) :
                1. 가우시안 커널 근사 $\phi(x) = e^x$
                2. ReLU 변형 $\phi(x) = \text{max}(x,0)$
    - 이러한 방식으로 복잡성이 $n^2$ -> $n$
- 쿼리 프로토타이핑과 모델 압축
    - ![qp_mc](images/11_04.png)
- 로우랭크 self attention

- 이외의 다양한 논문
    - [A SURVEY ON TRANSFORMERS IN NLP WITH FOCUS ON EFFICIENCY](https://arxiv.org/pdf/2406.16893)
    - [A Survey on Transformer Compression](https://arxiv.org/pdf/2402.05964)
    - [AWQ: ACTIVATION-AWARE WEIGHT QUANTIZATION FOR ON-DEVICE LLM COMPRESSION AND ACCELERATION](https://arxiv.org/pdf/2306.00978)
    - [DIFFERENTIAL TRANSFORMER](https://arxiv.org/pdf/2410.05258)

### 비전 트랜스 포머
- iGPT (image GPT)
    - 이미지의 픽셀을 시퀀스로 보고 GPT 모델구조와 자기회귀 를 이용해 훈련
- ViT
    - ![11_04](images/11_05.png)
    - 이미지의 BERT 스타일의 모델

### 테이블 데이터
- 쿼리와 결합한 트랜스 포머 아키텍쳐
    - ![11_04](images/11_06.png)


### 멀티 모달 트랜스 포머
- 스피치 투 텍스트
    - ![11_07](images/11_07.png)

- 비전 텍스트 모델
    - VQA
    - LayoutLM
    - DALLE
    - CLIP

- test for my self

[aya-expanse-8b](https://huggingface.co/CohereForAI/aya-expanse-8b)

In [ ]:
import json

with open("hf_key_token.json") as j:
    key = json.load(j)['hf_write_token']

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import huggingface_hub

huggingface_hub.login(token=key)

model_id = "CohereForAI/aya-expanse-8b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Format the message with the chat template
messages = [{"role": "user", "content": "Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz"}]
input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=100, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])
print(gen_text)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]